In [1]:
import numpy as np
from numpy import random 
import matplotlib.pyplot as plt
from physical_constants import RJ, Rsun, MJ, Gnewton
from ariel_wavelengths import ariel_wavelengths
from forward_model import compute_binned_modulations
plt.style.use('dark_background')

In [2]:
### Parameters for the planet and star:
temperature = 300  # temperature in Kelvin
pressure = 10 # pressure in bars!
planet_radius = RJ  # radius of the planet in meters
star_radius = Rsun  # radius of the star in meters
planet_mass = MJ    # mass of the planet in kg
g = Gnewton * planet_mass / (planet_radius**2)  # gravitational acceleration at the surface of the planet

### Define the atmospheric concentrations of the gases:
concentrations = {
    'CO2': 0.0, 
    'CO' : 0.0, 
    'NH3': 0.0, 
    'CH4': 0.0, 
    'H2O': 0.01,
}

modulations = compute_binned_modulations(
    concentrations=concentrations,
    temperature=temperature,
    pressure=pressure,
    planet_radius=planet_radius,
    star_radius=star_radius,
    g=g
)

### MCMC

In [3]:
from emilie_mcmc_utils import random_choice, get_log_likelihood, sample_proposal

In [4]:
# definition of bounds for fitted parameters
temp_min = 150  
temp_max = 250
# constants RJ and RSUN defined in physical_constants.py
# we take here plus and minus 10%
plrad_min = RJ - (0.1*RJ)
plrad_max = RJ + (0.1*RJ)
starrad_min = Rsun - (0.1*Rsun)
starrad_max = Rsun + (0.1*Rsun)

In [5]:
starting_state = random_choice(temp_min, temp_max, plrad_min, plrad_max, starrad_min, starrad_max)

Given the current parameter state and likelihood:

1. Propose a new candidate state.
2. Evaluate the new likelihood.
3. Accept or reject based on the Metropolis criterion:

$$r = \exp(\log \mathcal{L}_{\text{new}} - \log \mathcal{L}_{\text{current}})$$

If $r > \text{uniform}(0,1)$, accept the new state.

In [ ]:
# MCMC updater
def mcmc_updater(curr_state, curr_likeli, likelihood, proposal_distribution):
    proposal_state = proposal_distribution(curr_state)
    prop_likeli = likelihood(proposal_state)

    # Metropolis acceptance
    accept_ratio = np.exp(prop_likeli - curr_likeli)
    accept_threshold = np.random.uniform(0, 1)

    if accept_ratio > accept_threshold:
        return proposal_state, prop_likeli
    return curr_state, curr_likeli